In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np


In [2]:
# Sample English sentences and their ASL gloss translations
english_sentences = [
    "hello how are you",
    "what is your name",
    "my name is John"
]

asl_gloss_sentences = [
    "HELLO HOW YOU",
    "WHAT YOUR NAME",
    "MY NAME JOHN"
]

# Tokenize English sentences
english_tokenizer = Tokenizer(filters='')
english_tokenizer.fit_on_texts(english_sentences)
english_vocab_size = len(english_tokenizer.word_index) + 1

# Tokenize ASL gloss sentences
asl_gloss_tokenizer = Tokenizer(filters='')
asl_gloss_tokenizer.fit_on_texts(asl_gloss_sentences)
asl_gloss_vocab_size = len(asl_gloss_tokenizer.word_index) + 1

# Convert text sequences to integer sequences
english_sequences = english_tokenizer.texts_to_sequences(english_sentences)
asl_gloss_sequences = asl_gloss_tokenizer.texts_to_sequences(asl_gloss_sentences)

# Pad sequences to ensure uniform length
max_length = max(len(seq) for seq in english_sequences)
english_sequences_padded = pad_sequences(english_sequences, maxlen=max_length, padding='post')
asl_gloss_sequences_padded = pad_sequences(asl_gloss_sequences, maxlen=max_length, padding='post')


In [3]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(english_sequences_padded, asl_gloss_sequences_padded, test_size=0.2)

# Define the encoder-decoder model
embedding_dim = 16

encoder_inputs = tf.keras.layers.Input(shape=(max_length,))
encoder_embedding = tf.keras.layers.Embedding(english_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = tf.keras.layers.LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = tf.keras.layers.Input(shape=(max_length,))
decoder_embedding = tf.keras.layers.Embedding(asl_gloss_vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(asl_gloss_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [4]:
# Compile the model
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
# Train the model
model.fit([X_train, X_train], np.expand_dims(y_train, axis=-1), epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 2.1967 - val_accuracy: 0.2500 - val_loss: 2.1952
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5000 - loss: 2.1859 - val_accuracy: 0.2500 - val_loss: 2.1964
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5000 - loss: 2.1746 - val_accuracy: 0.2500 - val_loss: 2.1977
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5000 - loss: 2.1615 - val_accuracy: 0.2500 - val_loss: 2.1990
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5000 - loss: 2.1458 - val_accuracy: 0.2500 - val_loss: 2.2005
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5000 - loss: 2.1262 - val_accuracy: 0.2500 - val_loss: 2.2021
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5000 - loss: 2.1015 - val_accuracy: 0.2500 - val_loss: 2.2038
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5000 - loss: 2.0700 - val_accuracy: 0.2500 - val_los

In [6]:
# Evaluate the model
loss, accuracy = model.evaluate([X_test, X_test], np.expand_dims(y_test, axis=-1))
print("Test Accuracy:", accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.2500 - loss: 12.4978
Test Accuracy: 0.25


In [17]:
max_length_src = max(len(seq) for seq in english_sequences_padded)

input_sentence = "what is your name"

# Tokenize input sentence
input_sequence = english_tokenizer.texts_to_sequences([input_sentence])
input_sequence_padded = pad_sequences(input_sequence, maxlen=max_length_src, padding='post')

# Encode the input sequence
encoder_input = input_sequence_padded
reshaped_encoder_input = tf.expand_dims(encoder_input, axis=-1)
encoder_output, state_h_enc, state_c_enc = encoder_lstm(reshaped_encoder_input)
encoder_states = [state_h_enc, state_c_enc]

# Initialize decoder input sequence with start token
decoder_input = np.zeros((1, 1))
decoder_input[0, 0] = asl_gloss_tokenizer.word_index['<start>']

# Generate ASL gloss sequence
decoded_sentence = ''
while True:
    decoder_output, state_h_dec, state_c_dec = decoder_lstm(tf.expand_dims(decoder_input, axis=-1), initial_state=encoder_states)
    decoder_states = [state_h_dec, state_c_dec]
    decoder_prediction = decoder_dense(decoder_output)
    predicted_word_index = tf.argmax(decoder_prediction, axis=-1).numpy()[0][0]
    predicted_word = asl_gloss_tokenizer.index_word[predicted_word_index]
    if predicted_word == '<end>' or len(decoded_sentence.split()) > max_length_tar:
        break
    decoded_sentence += predicted_word + ' '
    # Update decoder input for the next iteration
    decoder_input = np.array([[predicted_word_index]])

print("Input Sentence:", input_sentence)
print("Predicted ASL Gloss:", decoded_sentence)

InvalidArgumentError: Exception encountered when calling LSTMCell.call().

[1m{{function_node __wrapped____MklMatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [1,1], In[1]: [16,1024] [Op:MatMul] name: [0m

Arguments received by LSTMCell.call():
  • inputs=tf.Tensor(shape=(1, 1), dtype=int32)
  • states=('tf.Tensor(shape=(1, 256), dtype=float32)', 'tf.Tensor(shape=(1, 256), dtype=float32)')
  • training=False